# AttGAN

# References
* [AttGAN: Facial Attribute Editing by Only Changing What You Want - IEEE Xplore ](https://ieeexplore.ieee.org/document/8718508)
* [AttGAN: Facial Attribute Editing by Only Changing What You Want - arXiv.org](https://arxiv.org/abs/1711.10678)
* [AttGAN-Tensorflow](https://github.com/LynnHo/AttGAN-Tensorflow)
* [AttGAN-PyTorch](https://github.com/elvisyjlin/AttGAN-PyTorch)



# Prerequisite
* Align CelebA dataset images using [align_images](https://github.com/look4pritam/TensorFlowExamples/blob/master/GAN/CelebA/align_images.ipynb) script.
* Download the preprocessed CelebA dataset using this [link](https://drive.google.com/file/d/1diaLDdB-dNMsPhJX0uco4155ghi4KMXK/view?usp=sharing).

# Install gdown Python package.

In [0]:
!pip install -U --no-cache-dir gdown

# Install Tensorflow-addons.
* InstanceNormalization

In [0]:
!pip install tensorflow-addons

# Import TensorFlow 2.x.

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

import matplotlib.pyplot as plot

print(tf.__version__)

# Set the root directory.

In [0]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

# Download the CelebA dataset from gdrive.

### Mount the gdrive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Copy aligned CelebA dataset.

In [0]:
!ls -al '/content/drive/My Drive/CelebA/img_align_celeba.tar.gz'

In [0]:
!cp '/content/drive/My Drive/CelebA/img_align_celeba.tar.gz' .

In [0]:
!ls -al

### Extract the dataset.

In [0]:
!tar -xzf img_align_celeba.tar.gz

In [0]:
!rm -rf img_align_celeba.tar.gz

In [0]:
!ls -al
!ls -al img_align_celeba
!ls -l img_align_celeba/images | wc -l

# Create dictionaries for facial attributes.
* Attributes to identifiers
* Identifiers to attributes

In [0]:
attributes_to_identifiers = {
    '5_o_Clock_Shadow': 0, 
    'Arched_Eyebrows': 1, 
    'Attractive': 2,       
    'Bags_Under_Eyes': 3,           
    'Bald': 4, 
    'Bangs': 5, 
    'Big_Lips': 6,           
    'Big_Nose': 7, 
    'Black_Hair': 8, 
    'Blond_Hair': 9, 
    'Blurry': 10,           
    'Brown_Hair': 11, 
    'Bushy_Eyebrows': 12, 
    'Chubby': 13,           
    'Double_Chin': 14, 
    'Eyeglasses': 15, 
    'Goatee': 16, 
    'Gray_Hair': 17, 
    'Heavy_Makeup': 18, 
    'High_Cheekbones': 19,          
    'Male': 20, 
    'Mouth_Slightly_Open': 21, 
    'Mustache': 22, 
    'Narrow_Eyes': 23, 
    'No_Beard': 24, 
    'Oval_Face': 25,           
    'Pale_Skin': 26, 
    'Pointy_Nose': 27, 
    'Receding_Hairline': 28,           
    'Rosy_Cheeks': 29, 
    'Sideburns': 30, 
    'Smiling': 31,           
    'Straight_Hair': 32, 
    'Wavy_Hair': 33, 
    'Wearing_Earrings': 34,           
    'Wearing_Hat': 35, 
    'Wearing_Lipstick': 36,           
    'Wearing_Necklace': 37, 
    'Wearing_Necktie': 38, 
    'Young': 39
    }

In [0]:
identifiers_to_attributes = {v: k for k, v in attributes_to_identifiers.items()}

# Prepare CelebA dataset in TensorFlow dataset format.

In [0]:
image_root_dir = 'img_align_celeba/images'
train_label_filename = 'img_align_celeba/train_label.txt'
val_label_filename = 'img_align_celeba/val_label.txt'

In [0]:
def create_celeba_dataset(image_root_dir, attribute_filename):
  image_names = np.genfromtxt(attribute_filename, dtype=str, usecols=0)
  image_filename_array = np.array([os.path.join(image_root_dir, image_name) for image_name in image_names])

  attributes_array = np.genfromtxt(attribute_filename, dtype=float, usecols=range(1, 41))    

  memory_data = (image_filename_array, attributes_array)  
  dataset = tf.data.Dataset.from_tensor_slices(memory_data)

  return(dataset)

In [0]:
train_dataset = create_celeba_dataset(image_root_dir, train_label_filename)

# Preprocess the dataset.

In [0]:
number_of_attributes = 40
image_load_shape = (143, 143, 3)
image_shape = (128, 128, 3)

In [0]:
buffer_size = 512
batch_size = 32

### Create test image.

In [0]:
def create_test_image(image_shape):
  test_image = np.random.rand(image_shape[0], image_shape[1], image_shape[2])
  return(test_image)

### Create test image batch.

In [0]:
def create_test_image_batch(image_shape):
  test_image_batch = np.random.rand(batch_size, image_shape[0], image_shape[1], image_shape[2])
  return(test_image_batch)

### Create test attributes.

In [0]:
def create_test_attributes():
  test_attributes = np.random.rand(number_of_attributes)
  return(test_attributes)

### Create test attribute batch.

In [0]:
def create_test_attribute_batch():
  test_attribute_batch = np.random.rand(batch_size, number_of_attributes)
  return(test_attribute_batch)

### Test created test attributes.

In [0]:
test_attributes = create_test_attributes()
print(test_attributes)

### Load image using filename.

In [0]:
def load_image(image_filename):
  input_image = tf.io.read_file(image_filename)
  input_image = tf.image.decode_jpeg(input_image, 3)
  return(input_image)

### Normalize the image to [-1, 1].

In [0]:
def normalize_image(image):
  image = tf.cast(image, tf.float32)
  image = tf.clip_by_value(image, 0, 255) / 127.5 - 1
  return(image)

### Test image normalization.

In [0]:
input_image = create_test_image(image_shape)
output_image = normalize_image(input_image)
print('input image shape',input_image.shape)
print('output image shape',output_image.shape)

### Random crop the image.

In [0]:
def random_crop(image):
  cropped_image = tf.image.random_crop(image, size=image_shape)
  return(cropped_image)

### Test random croping of image.

In [0]:
input_image = create_test_image(image_load_shape)
output_image = random_crop(input_image)
print('input image shape',input_image.shape)
print('output image shape',output_image.shape)

### Random jitter the image.

In [0]:
def random_jitter(image):  
  image = tf.image.resize(image, [image_load_shape[0], image_load_shape[1]],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)  
  image = random_crop(image)
  image = tf.image.random_flip_left_right(image)
  return(image)

### Test random jittering of image.

In [0]:
input_image = create_test_image(image_load_shape)
output_image = random_jitter(input_image)
print('input image shape',input_image.shape)
print('output image shape',output_image.shape)

### Preprocess train dataset.

In [0]:
def compute_attributes(attributes_array):
  attributes_array = (attributes_array + 1) // 2 
  attributes_array = attributes_array * 1.   
  return(attributes_array)

In [0]:
def preprocess_train_dataset(image_filename, attributes):  
  
  image = load_image(image_filename)
  image = random_jitter(image)
  image = normalize_image(image)

  attributes = compute_attributes(attributes)
  return(image, attributes)

### Preprocess the test dataset.

In [0]:
def preprocess_test_dataset(image_filename, attributes):    
  
  image = load_image(image_filename)
  image = tf.image.resize(image, [image_shape[0], image_shape[1]], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR) 
  image = normalize_image(image)

  attributes = compute_attributes(attributes)
  return(image, attributes)

### Preprocess dataset splits.

In [0]:
auto_tune = tf.data.experimental.AUTOTUNE

In [0]:
train_dataset = train_dataset.map(preprocess_train_dataset, num_parallel_calls=auto_tune)
train_dataset = train_dataset.shuffle(buffer_size)
train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
train_dataset = train_dataset.prefetch(auto_tune)

# Configuration parameters.

In [0]:
epochs = 100

In [0]:
adversarial_loss_mode = 'wgan'

In [0]:
d_gradient_penalty_weight = 10.0
d_attribute_loss_weight = 1.0

In [0]:
g_attribute_loss_weight = 10.0
g_reconstruction_loss_weight = 100.0

In [0]:
load_previous_weights = True
save_current_weights = True

In [0]:
epsilon = 1e-7

# Compute gradient penalty.

In [0]:
def gradient_penalty(discriminator, real_image, fake_image):
    sample_shape = [tf.shape(real_image)[0]] + [1] * (real_image.shape.ndims - 1)
    alpha = tf.random.uniform(shape=sample_shape, minval=0., maxval=1.)

    sample_image = real_image + alpha * (fake_image - real_image)
    sample_image.set_shape(real_image.get_shape().as_list())   
    
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(sample_image)

        predictions = discriminator(sample_image, training=False)
        if isinstance(predictions, tuple):
            predictions = predictions[0]

    gradients = tape.gradient(predictions, sample_image)[0]    
    gradients = tf.reshape(gradients, [tf.shape(gradients)[0], -1])
    norm = tf.norm(epsilon + gradients, axis=1)    
    gp_value = tf.reduce_mean((norm - 1.) ** 2)    

    return(gp_value)

# Create the optimizer.

*   Adam optimizer
*   Learning rate = 0.0002
*   β1 = 0.5
*   β2 = 0.999

In [0]:
encoder_optimizer = tf.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)
decoder_optimizer = tf.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)

discriminator_optimizer = tf.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)

# Create the encoder model.

In [0]:
def create_encoder_model(image_shape, encoder_dimension=64, downsamplings_layers=5):
  input_image = layers.Input(shape=image_shape, name='input_image')

  output_units = encoder_dimension    

  encoder_layer = input_image  
  feature_layers = []

  for layer_index in range(downsamplings_layers):
      layer_name = 'block-' + str(layer_index + 1) + '-'

      encoder_layer = layers.Conv2D(output_units, (4,4), strides=(2,2), padding='same', name=layer_name + 'conv')(encoder_layer)
      encoder_layer = layers.BatchNormalization(name=layer_name + 'bn')(encoder_layer)
      encoder_layer = layers.LeakyReLU(alpha=0.2, name=layer_name + 'features')(encoder_layer)

      feature_layers.append(encoder_layer)
      output_units = output_units * 2    
    
  # Create the encoder model.
  encoder_model = models.Model(inputs=input_image, outputs=feature_layers, name='encoder')  
 
  return(encoder_model)

### Test the encoder model.

In [0]:
encoder = create_encoder_model(image_shape, encoder_dimension=64, downsamplings_layers=5)
encoder.summary()

input_image = create_test_image_batch(image_shape)
output_features = encoder(input_image)
print('number of feature layers',len(output_features))
for block_index in range(len(output_features)):
    print('block-' +str(block_index) +' features shape -', output_features[block_index].shape)

# Concatenate features and attributes.

In [0]:
def concatenate(list_of_features, list_of_attributes, layer_name):
  list_of_features = list(list_of_features) if isinstance(list_of_features, (list, tuple)) else [list_of_features]
  list_of_attributes = list(list_of_attributes) if isinstance(list_of_attributes, (list, tuple)) else [list_of_attributes]
  for index, attributes in enumerate(list_of_attributes):
        attributes = tf.reshape(attributes, [-1, 1, 1, attributes.shape[-1]], name=layer_name + 'reshape')
        attributes = tf.tile(attributes, [1, list_of_features[0].shape[1], list_of_features[0].shape[2], 1], name=layer_name + 'tile')
        list_of_attributes[index] = attributes
  return tf.concat(list_of_features + list_of_attributes, axis=-1, name=layer_name + 'concat')

# Create the decoder model.

In [0]:
def create_decoder_model(encoder, number_of_attributes=40, decoder_dimension=64, upsamplings_layers=5, shortcut_layers=1, inject_layers=1):

  feature_layers = encoder.outputs
  number_of_feature_layers = len(feature_layers)

  input_features = []
  for layer_index in range(number_of_feature_layers):
    layer_name = 'block-' + str(layer_index + 1) + '-features'
    layer_shape = feature_layers[layer_index].shape
    layer_shape = layer_shape[1:]
    feature_layer = layers.Input(shape=layer_shape, name=layer_name) 
    input_features.append(feature_layer) 

  input_attributes = layers.Input(shape=number_of_attributes, name='input_attributes')  
  output_units = decoder_dimension

  layer_name = 'block-0-shortcut-'
  decoder_layer = concatenate(input_features[-1], input_attributes, layer_name=layer_name)
  for layer_index in range(upsamplings_layers - 1):
      layer_name = 'block-' + str(layer_index + 1) + '-'
      decoder_layer = layers.Conv2DTranspose(output_units, (4, 4), strides=(2,2), padding='same', name=layer_name+'convt')(decoder_layer)
      decoder_layer = layers.BatchNormalization(name=layer_name+'bn')(decoder_layer)
      decoder_layer = layers.LeakyReLU(alpha=0.2, name=layer_name+'lrelu')(decoder_layer)

      if (shortcut_layers > layer_index):
        shortcut_name = layer_name + 'shortcut-'
        decoder_layer = concatenate([decoder_layer, input_features[-2 - layer_index]], [], layer_name=shortcut_name)

      if (inject_layers > layer_index):
        inject_name = layer_name + 'inject-'
        decoder_layer = concatenate(decoder_layer, input_attributes, layer_name=inject_name)

      output_units = output_units * 2

  decoder_layer = layers.Conv2DTranspose(3, (4, 4), strides=(2,2), padding='same', name='block-5-convt')(decoder_layer)
  generated_image = layers.Activation('tanh', name='generated_image')(decoder_layer)

  # Create the decoder model.
  decoder_model = models.Model(inputs=[input_features, input_attributes], outputs=generated_image, name='decoder')

  return(decoder_model)

### Test the decoder model.

In [0]:
encoder = create_encoder_model(image_shape, encoder_dimension=64, downsamplings_layers=5)
encoder.summary()

decoder = create_decoder_model(encoder, number_of_attributes=40, decoder_dimension=64, upsamplings_layers=5, shortcut_layers=1, inject_layers=1)
decoder.summary()

input_image = create_test_image_batch(image_shape)
attributes = create_test_attribute_batch()

encoded_input = encoder(input_image)
decoded_output = decoder([encoded_input, attributes])
print(decoded_output.shape)

# Create the discriminator / classification model.

In [0]:
import tensorflow_addons as tfa

In [0]:
def create_discriminator_model(image_shape, number_of_attributes=40, discriminator_dimension=64, dense_dimension=1024, downsamplings_layers=5):
    input_image = layers.Input(shape=image_shape, name='input_image')  
    input_attributes = layers.Input(shape=number_of_attributes, name='input_attributes')  

    output_units = discriminator_dimension
    input_layer = input_image
    
    for layer_index in range(downsamplings_layers): 
        input_layer = layers.Conv2D(output_units, (4,4), strides=(2,2), padding='same')(input_layer)         
        input_layer = tfa.layers.InstanceNormalization()(input_layer)
        input_layer = layers.LeakyReLU(alpha=0.2)(input_layer)

        output_units = output_units * 2

    input_layer = layers.Flatten()(input_layer)

    discriminator_output = layers.Dense(dense_dimension)(input_layer) 
    discriminator_output = tfa.layers.InstanceNormalization()(discriminator_output)     
    discriminator_output = layers.LeakyReLU(alpha=0.2)(discriminator_output)
    discriminator_output = layers.Dense(1, activation=None)(discriminator_output)    
      
    attribute_output = layers.Dense(dense_dimension)(input_layer)  
    attribute_output = tfa.layers.InstanceNormalization()(attribute_output)         
    attribute_output = layers.LeakyReLU(alpha=0.2)(attribute_output)
    attribute_output = layers.Dense(number_of_attributes, activation=None)(attribute_output)    

    # Create the discriminator model.
    discriminator_model = models.Model(inputs=[input_image, input_attributes], outputs=[discriminator_output, attribute_output], name='discriminator')

    return(discriminator_model)    

### Test the discriminator model.

In [0]:
input_image = create_test_image_batch(image_shape)

discriminator = create_discriminator_model(image_shape, number_of_attributes=40, discriminator_dimension=64, dense_dimension=1024, downsamplings_layers=5)
discriminator_prediction, attribute_prediction = discriminator(input_image)

print('discriminator prediction shape', discriminator_prediction.shape)
print('attribute prediction shape', attribute_prediction.shape)

# Create adversarial loss functions.
*   Generator loss function
*   Discriminator loss function

## WGAN loss functions.
*   Generator loss function
*   Discriminator loss function

In [0]:
def wgan_loss_functions():
    def discriminator_loss_function(real_logit, fake_logit):
        real_loss = - tf.reduce_mean(real_logit)
        fake_loss = tf.reduce_mean(fake_logit)
        return(real_loss, fake_loss)

    def generator_loss_function(fake_logit):
        fake_loss = - tf.reduce_mean(fake_logit)
        return(fake_loss)

    return(discriminator_loss_function, generator_loss_function)

In [0]:
def adversarial_loss_functions(adversarial_loss_mode):
  if(adversarial_loss_mode == 'wgan'):
    return(wgan_loss_functions())
  else:
    return(wgan_loss_functions())

# Create different models and loss functions.
* Encoder model
* Decoder model
* Discriminator model
* Discriminator loss function
* Generator loss function

In [0]:
encoder = create_encoder_model(image_shape, encoder_dimension=64, downsamplings_layers=5)
decoder = create_decoder_model(encoder, number_of_attributes=40, decoder_dimension=64,)
discriminator = create_discriminator_model(image_shape, number_of_attributes=40, discriminator_dimension=64, dense_dimension=1024, downsamplings_layers=5)

discriminator_loss_function, generator_loss_function = adversarial_loss_functions(adversarial_loss_mode)

# Create the generator model.

In [0]:
input_image = layers.Input(shape=image_shape, name='input_image')  
input_attributes = layers.Input(shape=number_of_attributes, name='input_attributes')

encoded_features = encoder(input_image)
generated_image = decoder([encoded_features, input_attributes])

generator_model = models.Model(inputs=[input_image, input_attributes], outputs=generated_image, name='generator')
generator_model.summary()

### Test the generator model.

In [0]:
input_image = create_test_image_batch(image_shape)
attributes = create_test_attribute_batch()
generated_image = generator_model([input_image, attributes])

print('input image shape',input_image.shape)
print('generated image shape',generated_image.shape)

### Load previous model weights.
* Encoder model weights
* Decoder model weights
* Discriminator model weights



In [0]:
import os 

def encoder_filename():
  return('encoder.h5')

def encoder_gdrive_filename(weight_root_dir='/content/drive/My Drive/models/AttGAN/'):    
  return(os.path.join(weight_root_dir, encoder_filename()))

def decoder_filename():
  return('decoder.h5')

def decoder_gdrive_filename(weight_root_dir='/content/drive/My Drive/models/AttGAN/'):    
  return(os.path.join(weight_root_dir, decoder_filename()))

def discriminator_filename():
  return('discriminator.h5')

def discriminator_gdrive_filename(weight_root_dir='/content/drive/My Drive/models/AttGAN/'):  
  return(os.path.join(weight_root_dir, discriminator_filename()))

In [0]:
if(load_previous_weights):
  encoder.load_weights(encoder_gdrive_filename())
  decoder.load_weights(decoder_gdrive_filename())
  discriminator.load_weights(discriminator_gdrive_filename())

# Compute the generator loss.

In [0]:
def compute_generator_loss(input_image, input_attributes):

  target_attributes = tf.random.shuffle(input_attributes)

  scaled_input_attributes = input_attributes * 2. - 1.
  scaled_target_attributes = target_attributes * 2. - 1.

  # Generator
  input_features = encoder(input_image, training=True)
  reconstructed_image = decoder([input_features, scaled_input_attributes], training=True)
  fake_image = decoder([input_features, scaled_target_attributes], training=True)

  # Discriminator
  fake_image_prediction, fake_image_attributes = discriminator(fake_image, training=False)

  fake_image_prediction_loss = generator_loss_function(fake_image_prediction)
  fake_image_attributes_loss = tf.compat.v1.losses.sigmoid_cross_entropy(target_attributes, fake_image_attributes)  
  
  input_image_reconstruction_loss = tf.compat.v1.losses.absolute_difference(input_image, reconstructed_image)
   
  generator_loss = (  fake_image_prediction_loss 
                    + fake_image_attributes_loss * g_attribute_loss_weight 
                    + input_image_reconstruction_loss * g_reconstruction_loss_weight
                    )  

  return(generator_loss)

### Test computation of the generator loss.

In [0]:
images, attributes = next(iter(train_dataset))
loss = compute_generator_loss(images, attributes)
print(loss.numpy())

# Compute the discriminator loss.

In [0]:
def compute_discriminator_loss(input_image, input_attributes):

  target_attributes = tf.random.shuffle(input_attributes)

  scaled_input_attributes = input_attributes * 2. - 1.
  scaled_target_attributes = target_attributes * 2. - 1.

  # Generate
  input_features = encoder(input_image, training=False)  
  fake_image = decoder([input_features, scaled_target_attributes], training=False)

  # Discriminate
  input_image_prediction, input_image_attributes = discriminator(input_image, training=True)
  fake_image_prediction, fake_image_attributes = discriminator(fake_image, training=True)

  # Discriminator losses
  input_image_gan_loss, fake_image_gan_loss = discriminator_loss_function(input_image_prediction, fake_image_prediction)  
  gradient_penalty_value = gradient_penalty(discriminator, input_image, fake_image)      

  input_image_attributes_loss = tf.compat.v1.losses.sigmoid_cross_entropy(input_attributes, input_image_attributes)  

  discriminator_loss = (  input_image_gan_loss 
                        + fake_image_gan_loss 
                        + gradient_penalty_value * d_gradient_penalty_weight 
                        + input_image_attributes_loss * d_attribute_loss_weight
                        )  
  
  return(discriminator_loss)

### Test computation of the discriminator loss.

In [0]:
images, attributes = next(iter(train_dataset))
loss = compute_discriminator_loss(images, attributes)
print(loss.numpy())

# Train the model.

In [0]:
model_loss_frequency = 500
model_save_frequency = 2000

In [0]:
def save_models():
  if(save_current_weights):  
    encoder.save_weights(encoder_gdrive_filename())        
    decoder.save_weights(decoder_gdrive_filename())      
    discriminator.save_weights(discriminator_gdrive_filename())  

In [0]:
def train(train_dataset, epochs=100):  

  for epoch in range(epochs):

    batch_index = 0    
    for dataset_batch in train_dataset:  
      batch_index = batch_index + 1    

      images, attributes = dataset_batch
      
      if(batch_index%6 == 0):

        with tf.GradientTape(watch_accessed_variables=False) as encoder_tape, tf.GradientTape(watch_accessed_variables=False) as decoder_tape:
          encoder_tape.watch(encoder.trainable_variables)
          decoder_tape.watch(decoder.trainable_variables)
          generator_loss = compute_generator_loss(images, attributes)

        enoder_gradients = encoder_tape.gradient(generator_loss, encoder.trainable_variables)
        decoder_gradients = decoder_tape.gradient(generator_loss, decoder.trainable_variables)

        encoder_optimizer.apply_gradients(zip(enoder_gradients, encoder.trainable_variables))
        decoder_optimizer.apply_gradients(zip(decoder_gradients, decoder.trainable_variables))

      else:
        with tf.GradientTape(watch_accessed_variables=False) as discriminator_tape:
          discriminator_tape.watch(discriminator.trainable_variables)
          discriminator_loss = compute_discriminator_loss(images, attributes)

        discriminator_gradients = discriminator_tape.gradient(discriminator_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))        

      if(batch_index%model_loss_frequency == 0):
        print('epoch -', epoch, 'generator loss -', generator_loss.numpy(), 'discriminator loss -', discriminator_loss.numpy())

      if(batch_index%model_save_frequency == 0):
        save_models()

    # Save model weights at the end of epoch.
    save_models()
        

In [0]:
train(train_dataset, epochs=10)

In [0]:
save_models()